In [52]:
import tensorflow as tf
import tensorflow_gnn as tfgnn
from importlib import reload
from sys import modules
from schema import TYPE_SPEC
from generators import load_dataset_from_data

In [53]:
reload(modules["schema"])
from schema import TYPE_SPEC
spec = TYPE_SPEC["mnist_graph"]

In [60]:
input = tf.keras.layers.Input(type_spec=spec)
gnn = tfgnn.keras.ConvGNNBuilder(
    lambda edge: tfgnn.keras.layers.SimpleConvolution(
        tf.keras.layers.Dense(128)
    ),
    lambda node: tfgnn.keras.layers.NextStateFromConcat(
        tf.keras.layers.Dense(64),
    ),
    )
hidden = gnn.Convolve()(input)
hidden = tfgnn.keras.layers.Readout(node_set_name="pixel")(hidden)
output = tf.keras.layers.Dense(10, activation="softmax")(hidden)

model = tf.keras.Model(input, output, name = "gnn")
model.summary()

Model: "gnn"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_9 (InputLayer)        [()]                      0         
                                                                 
 graph_update_8 (GraphUpdate  ()                       259264    
 )                                                               
                                                                 
 readout_8 (Readout)         (None, 64)                0         
                                                                 
 dense_8 (Dense)             (None, 10)                650       
                                                                 
Total params: 259,914
Trainable params: 259,914
Non-trainable params: 0
_________________________________________________________________


In [55]:
model.compile(optimizer="adam", loss=tf.keras.losses.categorical_crossentropy, metrics=["accuracy"])

In [56]:
reload(modules["generators"])
from generators import load_dataset_from_data

(trn_img, trn_lbl), _ = tf.keras.datasets.mnist.load_data()
trn_img = trn_img/255

In [58]:
trn_graph = load_dataset_from_data(trn_img, trn_lbl, 1, spec)
print(type(trn_graph))
# print(tfgnn.graph_tensor_to_values(list(trn_graph.take(1))[0]))

<class 'tensorflow.python.data.ops.dataset_ops.MapDataset'>


In [59]:
model.fit(trn_graph, epochs=10)

Epoch 1/10


InvalidArgumentError: Graph execution error:

Detected at node 'gnn/graph_update_7/node_set_update/simple_convolution/edge_set_update/GatherV2_1' defined at (most recent call last):
    File "/usr/local/lib/python3.9/runpy.py", line 197, in _run_module_as_main
      return _run_code(code, main_globals, None,
    File "/usr/local/lib/python3.9/runpy.py", line 87, in _run_code
      exec(code, run_globals)
    File "/home/thomas/Documents/tfgnn-mnist/env/lib/python3.9/site-packages/ipykernel_launcher.py", line 16, in <module>
      app.launch_new_instance()
    File "/home/thomas/Documents/tfgnn-mnist/env/lib/python3.9/site-packages/traitlets/config/application.py", line 846, in launch_instance
      app.start()
    File "/home/thomas/Documents/tfgnn-mnist/env/lib/python3.9/site-packages/ipykernel/kernelapp.py", line 677, in start
      self.io_loop.start()
    File "/home/thomas/Documents/tfgnn-mnist/env/lib/python3.9/site-packages/tornado/platform/asyncio.py", line 199, in start
      